In [1]:
import os
import pandas as pd
import numpy as np
import ast
from typing import TypedDict, List
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END

In [2]:
df_hospitals = pd.read_csv('hospitals_kolkata.csv')
df_doctors = pd.read_csv('doctors_kolkata.csv')

In [3]:

print(f"Loaded {len(df_hospitals)} hospitals and {len(df_doctors)} doctors.")

Loaded 15 hospitals and 365 doctors.


In [4]:
import math
import os
import uuid
import pandas as pd
import requests
import datetime as dt
from dataclasses import dataclass
from typing import List, Optional, Dict, Any
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# --- Data Models for Ambulance ---
@dataclass
class EmergencyRequest:
    request_id: str
    lat: float
    lng: float
    severity_level: str
    emergency_type: str
    is_child: bool
    timestamp: str
    zone_id: Optional[str] = None

@dataclass
class AmbulanceCandidate:
    ambulance_id: str
    distance_km: float
    eta_min: float
    score: float
    meta: Dict[str, Any]

# --- Helper Function ---
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# --- Ambulance Agent Class ---
class AmbulanceAgent:
    def __init__(self, csv_path: str = "ambulances_kolkata.csv", google_api_key: Optional[str] = None, mode: str = "fast"):
        self.csv_path = csv_path
        self.mode = mode.lower()
        self.google_api_key = google_api_key or os.getenv("GOOGLE_MAPS_API_KEY")
        
        # Load Data
        try:
            self.ambulances_df = pd.read_csv(csv_path)
            # Basic data cleaning for booleans
            bool_cols = ["can_handle_critical", "pediatric_capable"]
            for c in bool_cols:
                if c in self.ambulances_df.columns and self.ambulances_df[c].dtype == object:
                    self.ambulances_df[c] = self.ambulances_df[c].astype(str).str.lower().isin(["true", "1", "yes"])
        except FileNotFoundError:
            print(f"❌ Error: {csv_path} not found. Please ensure the CSV is in the folder.")
            self.ambulances_df = pd.DataFrame() # Empty DF to prevent crash on init

    def _get_eta_for_row(self, row) -> float:
        return float(row.get("eta_min_live", row["eta_min_approx"]))

    def handle_new_emergency(self, emergency: EmergencyRequest, top_k: int = 3) -> Optional[Dict[str, Any]]:
        # 1. Filter & Score
        candidates = self._find_best_ambulances(emergency, top_k=top_k)
        if not candidates:
            return None
        
        # 2. Simulate Acceptance (Pick top 1 for simplicity in this agent)
        chosen = candidates[0]
        
        # 3. Route
        route_info = self._compute_route(chosen, emergency)
        
        return {
            "assigned_ambulance_id": chosen.ambulance_id,
            "eta_min": round(chosen.eta_min, 1),
            "route_info": route_info
        }

    def _find_best_ambulances(self, emergency: EmergencyRequest, top_k: int = 3) -> List[AmbulanceCandidate]:
        if self.ambulances_df.empty: return []
        df = self.ambulances_df.copy()
        
        # Filter Status
        df = df[df["status"].isin(["IDLE", "AT_HOSPITAL"])]
        
        # Calculate Distance & Approx ETA
        df["distance_km"] = df.apply(lambda r: haversine(r["current_lat"], r["current_lng"], emergency.lat, emergency.lng), axis=1)
        df["eta_min_approx"] = (df["distance_km"] / 25.0) * 60.0 # 25 km/h avg speed
        
        # Simple Scoring (Distance + Type Match)
        df["score"] = df.apply(lambda r: 1.0 / (r["distance_km"] + 0.1), axis=1) 
        
        # Sort
        df = df.sort_values(by=["score"], ascending=False).head(top_k)
        
        results = []
        for _, row in df.iterrows():
            results.append(AmbulanceCandidate(
                ambulance_id=row["ambulance_id"],
                distance_km=row["distance_km"],
                eta_min=row["eta_min_approx"],
                score=row["score"],
                meta=row.to_dict()
            ))
        return results

    def _compute_route(self, chosen: AmbulanceCandidate, emergency: EmergencyRequest):
        # Fallback to simple logic if no Google Key
        return {
            "from_lat": chosen.meta["current_lat"],
            "from_lng": chosen.meta["current_lng"],
            "to_lat": emergency.lat,
            "to_lng": emergency.lng,
            "estimated_eta_min": chosen.eta_min
        }

# Instantiate the Agent globally so we don't reload CSV every time
# Make sure 'ambulances_kolkata.csv' exists in your folder!
ambulance_service = AmbulanceAgent(csv_path="ambulances_kolkata.csv", mode="fast")
print("✅ Ambulance Agent Initialized")

✅ Ambulance Agent Initialized


In [5]:
def hospital_ann_score(features):
    """
    Simulates a trained Neural Network forward pass.
    Inputs: [Normalized_Distance, ICU_Free_Norm, Staff_Norm, Trauma_Center_Binary]
    """
    # Weights adjusted for your new features:
    # High priority on Trauma Center capability (0.9) and Distance (-0.6)
    weights = np.array([-0.6, 0.3, 0.1, 0.9]) 
    bias = 0.1
    score = np.dot(features, weights) + bias
    return max(0, score)

In [6]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

def setup_doctor_knowledge_base(df_doctors):
    print("--- Building Doctor Knowledge Base ---")
    docs = []

    for _, row in df_doctors.iterrows():
        content = (
            f"Doctor: {row['name']} (ID: {row['doctor_id']}). "
            f"Level: {row['seniority_level']}. "
            f"Specialty: {row['specialty']} ({row['sub_specialty']}). "
            f"Experience: {row['years_experience']} years. "
            f"Languages: {row['languages_spoken']}. "
            f"On Call: {row['on_call']}. "
            f"Prefers Trauma: {row['prefers_trauma_cases']}."
        )

        meta = {
            "hospital_id": row["hospital_id"],
            "doctor_id": row["doctor_id"],
        }

        docs.append(Document(page_content=content, metadata=meta))

    # 🔹 choose your model here:
    # "BAAI/bge-small-en"  = fast, English
    # "BAAI/bge-m3"        = multilingual, very good
    model_name = "BAAI/bge-small-en"   # or "BAAI/bge-m3"

    # BGE likes cosine similarity + normalized vectors
    encode_kwargs = {"normalize_embeddings": True}

    embeddings = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cpu"},   # change to "cuda" if you have GPU
        encode_kwargs=encode_kwargs,
    )

    vector_store = FAISS.from_documents(docs, embeddings)
    return vector_store

# usage:
vector_db = setup_doctor_knowledge_base(df_doctors)


--- Building Doctor Knowledge Base ---


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_2244\2442373929.py:35: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(


In [7]:
vector_db.save_local("doctor_faiss_index")

In [8]:
class AgentState(TypedDict):
    # Inputs
    accident_location: tuple 
    accident_type: str 
    
    # Ambulance Agent Outputs (NEW)
    assigned_ambulance: str
    ambulance_eta: float
    
    # Hospital Agent Outputs
    selected_hospital_id: str
    selected_hospital_name: str
    
    # Doctor Agent Outputs
    final_plan: str

In [9]:
def ambulance_node(state: AgentState):
    print(f"\n--- [Ambulance Agent] Dispatching for: {state['accident_type']} ---")
    
    lat, lng = state['accident_location']
    
    # Create the request object required by your class
    req = EmergencyRequest(
        request_id=str(uuid.uuid4()),
        lat=lat,
        lng=lng,
        severity_level="CRITICAL", # We assume critical for this workflow
        emergency_type=state['accident_type'],
        is_child=False,
        timestamp=dt.datetime.now().isoformat()
    )
    
    # Call the agent logic
    result = ambulance_service.handle_new_emergency(req)
    
    if result:
        amb_id = result['assigned_ambulance_id']
        eta = result['eta_min']
        print(f"--- [Ambulance Agent] Dispatched {amb_id} (ETA: {eta} mins) ---")
        return {
            "assigned_ambulance": amb_id,
            "ambulance_eta": eta
        }
    else:
        print("--- [Ambulance Agent] No ambulance found! ---")
        return {
            "assigned_ambulance": "NONE",
            "ambulance_eta": 999.0
        }

In [10]:
def hospital_agent(state: AgentState):
    """
    Finds best hospital using 'icu_beds_free', 'facility_trauma_center', etc.
    """
    print(f"\n--- [Hospital Agent] Scanning for: {state['accident_type']} ---")
    
    user_lat, user_lng = state['accident_location']
    scored_hospitals = []

    for _, row in df_hospitals.iterrows():
        # 1. Distance
        dist = np.sqrt((row['lat'] - user_lat)**2 + (row['lng'] - user_lng)**2)
        norm_dist = min(dist * 10, 1.0)
        
        # 2. Features (Mapped to your new headers)
        # ICU Availability
        icu_free = row['icu_beds_free'] if pd.notna(row['icu_beds_free']) else 0
        norm_icu = min(icu_free / 20, 1.0) # Normalize (assuming 20 is a good number)
        
        # Staffing
        staff_count = row['staff_doctors_planned'] if pd.notna(row['staff_doctors_planned']) else 0
        norm_staff = min(staff_count / 50, 1.0)
        
        # Trauma Center Capability
        # Check if accident is trauma-related AND facility has trauma center
        is_trauma_accident = 'CRASH' in state['accident_type'].upper() or 'TRAUMA' in state['accident_type'].upper()
        
        # Using the boolean column directly
        has_trauma_center = 1.0 if row['facility_trauma_center'] == True else 0.0
        
        # If it's a trauma accident, having a trauma center is crucial (1.0). If not, it's neutral (0.0).
        trauma_score = has_trauma_center if is_trauma_accident else 0.5
        
        # 3. ANN Scoring
        features = np.array([norm_dist, norm_icu, norm_staff, trauma_score])
        score = hospital_ann_score(features)
        
        scored_hospitals.append({
            "id": row['hospital_id'],
            "name": row['hospital_name'], 
            "score": score,
            "dist": dist
        })
    
    # Sort and Select
    best_hospital = sorted(scored_hospitals, key=lambda x: x['score'], reverse=True)[0]
    
    print(f"--- [Hospital Agent] Selected: {best_hospital['name']} (Score: {best_hospital['score']:.2f}) ---")
    return {
        "selected_hospital_id": best_hospital['id'],
        "selected_hospital_name": best_hospital['name']
    }

In [11]:
import os
import json
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings  # Updated import for modern LangChain
from dotenv import load_dotenv  # <--- NEW IMPORT

# Load variables from .env file into the environment
load_dotenv()

# --- 1. LOAD THE SAVED VECTOR DATABASE ---
# We must use the exact same model config used during the save step
print("--- Loading Vector Database from Disk ---")

embedding_model_name = "BAAI/bge-small-en"
encode_kwargs = {"normalize_embeddings": True}

# Initialize the embedding model
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"device": "cpu"}, 
    encode_kwargs=encode_kwargs
)

# Load the FAISS index from your folder
try:
    vector_db = FAISS.load_local(
        "doctor_faiss_index",  # Folder name containing index.faiss and index.pkl
        embeddings, 
        allow_dangerous_deserialization=True # Required for loading pickle files you created
    )
    print("✅ Vector DB loaded successfully.")
except Exception as e:
    print(f"❌ Error loading Vector DB: {e}")
    print("Ensure the folder 'doctor_faiss_index' exists in this directory.")
    # Stop execution if DB fails to load, as the agent depends on it
    raise e 


# --- 2. DOCTOR AGENT FUNCTION ---

def doctor_agent(state: AgentState):
    """
    Finds doctor using the loaded RAG Vector DB and ensures Hospital Name is in the output.
    """
    hosp_id = state['selected_hospital_id']
    hosp_name = state['selected_hospital_name'] 
    
    print(f"--- [Doctor Agent] Querying staff at {hosp_id} ({hosp_name}) ---")
    
    # 1. RETRIEVAL
    # Use the loaded vector_db to create a retriever
    retriever = vector_db.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3,
            "filter": {"hospital_id": hosp_id}, # Strict filter for the selected hospital
        },
    )
    
    query = f"""
    Accident: {state['accident_type']}
    Find a doctor with high seniority, trauma preference, and On Call=True.
    """
    
    # Fetch documents
    # invoke() returns a list of Documents in standard LangChain retrievers
    docs = retriever.invoke(query)
    
    # Debug: Verify we actually found doctors
    if not docs:
        print(f"⚠️ Warning: No doctors found for hospital {hosp_id} in Vector DB.")

    # 2. GENERATION (GROQ)
    llm = ChatGroq(
        temperature=0,
        model="llama-3.1-8b-instant",
        api_key=os.getenv("GROQ_API") # Your Key
    )
    
    context_text = "\n\n".join([d.page_content for d in docs])
    
    prompt = f"""
    Emergency Type: {state['accident_type']}
    Selected Hospital: {hosp_name}

    Candidate Doctors (Retrieved from Database):
    {context_text}

    Task:
    - Pick the BEST doctor based on availability and specialty.
    - Return a VALID JSON object (no markdown formatting, just raw JSON) with this structure: 
      {{
        "recommended_hospital": "{hosp_name}", 
        "target_doctor": "Name",
        "doctor_specialty": "Specialty",
        "status": "ON_CALL",
        "action": "Prepare OR",
        "reason": "One short sentence explanation"
      }}
    """
    
    response = llm.invoke(prompt)
    return {"final_plan": response.content}

--- Loading Vector Database from Disk ---
✅ Vector DB loaded successfully.


In [12]:
from langgraph.graph import StateGraph, END

# Initialize Graph
workflow = StateGraph(AgentState)

# Add Nodes
workflow.add_node("find_ambulance", ambulance_node)   # <--- NEW NODE
workflow.add_node("find_hospital", hospital_agent)    # Existing node
workflow.add_node("find_doctor", doctor_agent)        # Existing node

# Define Edges
# 1. Start at Ambulance
workflow.set_entry_point("find_ambulance")

# 2. Ambulance -> Hospital
workflow.add_edge("find_ambulance", "find_hospital")

# 3. Hospital -> Doctor
workflow.add_edge("find_hospital", "find_doctor")

# 4. Doctor -> End
workflow.add_edge("find_doctor", END)

# Compile
app = workflow.compile()
print("✅ New Workflow with Ambulance Agent compiled.")

✅ New Workflow with Ambulance Agent compiled.


In [13]:
inputs = {
    "accident_location": (22.5629, 88.3962), # Example location in Kolkata
    "accident_type": "Multiple vehicle collision - Trauma"
}

print(f"🚨 ALERT: {inputs['accident_type']} at {inputs['accident_location']}")
print("===============================================================")

result = app.invoke(inputs)

print("\n================ FINAL ORCHESTRATION REPORT ================")
print(f"🚑 AMBULANCE: {result.get('assigned_ambulance')} (ETA: {result.get('ambulance_eta')} min)")
print(f"🏥 HOSPITAL:  {result.get('selected_hospital_name')}")
print(f"👨‍⚕️ PLAN:      {result.get('final_plan')}")

🚨 ALERT: Multiple vehicle collision - Trauma at (22.5629, 88.3962)

--- [Ambulance Agent] Dispatching for: Multiple vehicle collision - Trauma ---
--- [Ambulance Agent] Dispatched AMB_020 (ETA: 3.5 mins) ---

--- [Hospital Agent] Scanning for: Multiple vehicle collision - Trauma ---
--- [Hospital Agent] Selected: CityCare Hospital WA (Score: 0.83) ---
--- [Doctor Agent] Querying staff at HOSP_004 (CityCare Hospital WA) ---

================ FINAL ORCHESTRATION REPORT ================
🚑 AMBULANCE: AMB_020 (ETA: 3.5 min)
🏥 HOSPITAL:  CityCare Hospital WA
👨‍⚕️ PLAN:      ```json
{
  "recommended_hospital": "CityCare Hospital WA",
  "target_doctor": "Sara Roy",
  "doctor_specialty": "TRAUMA (HEAD_INJURY_FOCUS)",
  "status": "ON_CALL",
  "action": "Prepare OR",
  "reason": "Sara Roy is available and has a focus on head injury trauma."
}
```

This JSON object recommends Sara Roy as the best doctor based on her availability (ON_CALL) and specialty (TRAUMA with a focus on HEAD_INJURY).
